In [5]:
import pandas as pd
import ast
import requests
import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from collections import Counter
import networkx as nx
import matplotlib.pyplot as plt

# Loading and execution

In [4]:
entities_df = pd.read_csv('C:/Users/maria/OneDrive/Documentos/Thesis/dataset_3/papers_phylogeny_ordered.csv', encoding='utf-8')
entities_df

,fileMain_s,TaxoEntitiesMD2_GBIF,TaxoEntities_Count,Species_Count,kingdom_tuple_Count,phylum_tuple_Count,class_tuple_Count,order_tuple_Count,family_tuple_Count,genus_tuple_Count,species_tuple_Count
0,https://hal.inrae.fr/hal-03461601/document,"[[('GBIF:3183002', 'clove', 0.7634411454200745...","Counter({('GBIF:3183002', 'clove'): 11, ('GBIF...","[Counter({('GBIF:3183002', 'clove'): 11}), Cou...","[(('GBIF:6', 'Plantae'), 141), (('GBIF:1', 'An...","[(('GBIF:7707728', 'Tracheophyta'), 141), (('G...","[(('GBIF:220', 'Magnoliopsida'), 73), (('GBIF:...","[(('GBIF:627', 'Zingiberales'), 58), (('GBIF:1...","[(('GBIF:4686', 'Musaceae'), 56), (('GBIF:7163...","[(('GBIF:2760990', 'Musa'), 56), (('GBIF:23410...","[(('GBIF:5204104', 'Albula vulpes'), 28), (('G..."
1,https://hal.science/hal-03829927/document,"[[('GBIF:9065640', 'Tyto novaehollandiae novae...","Counter({('GBIF:5219788', 'Pseudomys higginsi'...","[Counter({('GBIF:5232358', 'Tyto tenebricosa')...","[(('GBIF:1', 'Animalia'), 186), (('GBIF:6', 'P...","[(('GBIF:44', 'Chordata'), 177), (('GBIF:54', ...","[(('GBIF:359', 'Mammalia'), 148), (('GBIF:212'...","[(('GBIF:1459', 'Rodentia'), 54), (('GBIF:736'...","[(('GBIF:5510', 'Muridae'), 46), (('GBIF:5452'...","[(('GBIF:2438300', 'Pseudomys'), 22), (('GBIF:...","[(('GBIF:2440062', 'Pseudocheirus peregrinus')..."
2,https://hal.science/hal-02350603/document,[],Counter(),[],[],[],[],[],[],[],[]
3,https://shs.hal.science/halshs-02937670/document,"[[('GBIF:5786457', 'Sateré Marmoset', 0.771187...","Counter({('GBIF:3189949', 'guarana'): 33, ('GB...","[Counter({('GBIF:5786457', 'Sateré Marmoset'):...","[(('GBIF:6', 'Plantae'), 112), (('GBIF:1', 'An...","[(('GBIF:7707728', 'Tracheophyta'), 112), (('G...","[(('GBIF:220', 'Magnoliopsida'), 112), (('GBIF...","[(('GBIF:933', 'Sapindales'), 106), (('GBIF:79...","[(('GBIF:6657', 'Sapindaceae'), 105), (('GBIF:...","[(('GBIF:3189948', 'Paullinia'), 105), (('GBIF...","[(('GBIF:3189949', 'Paullinia cupana'), 105), ..."
4,https://cnam.hal.science/hal-04080723/document,"[[('GBIF:4494312', 'Spilomena beata Blüthgen',...","Counter({('GBIF:4494312', 'Spilomena beata Blü...","[Counter({('GBIF:4494312', 'Spilomena beata Bl...","[(('GBIF:1', 'Animalia'), 3), (('GBIF:6', 'Pla...","[(('GBIF:54', 'Arthropoda'), 3), (('GBIF:77077...","[(('GBIF:216', 'Insecta'), 3), (('GBIF:220', '...","[(('GBIF:1457', 'Hymenoptera'), 3), (('GBIF:93...","[(('GBIF:7906', 'Crabronidae'), 3), (('GBIF:23...","[(('GBIF:1346854', 'Spilomena'), 3), (('GBIF:7...","[(('GBIF:4494312', 'Spilomena beata'), 3), (('..."
...,...,...,...,...,...,...,...,...,...,...,...
1151,https://hal.sorbonne-universite.fr/hal-0295053...,"[[('GBIF:2368377', 'Oryzias latipes', 1.0)], [...","Counter({('GBIF:5073713', 'Drosophila melanoga...","[Counter({('GBIF:2368377', 'Oryzias latipes'):...","[(('GBIF:1', 'Animalia'), 124)]","[(('GBIF:44', 'Chordata'), 109), (('GBIF:54', ...","[((None, nan), 97), (('GBIF:359', 'Mammalia'),...","[(('GBIF:1313', 'Salmoniformes'), 34), (('GBIF...","[(('GBIF:8615', 'Salmonidae'), 34), (('GBIF:55...","[(('GBIF:2351426', 'Oncorhynchus'), 34), (('GB...","[(('GBIF:5204019', 'Oncorhynchus mykiss'), 32)..."
1152,https://u-picardie.hal.science/hal-03619105/do...,"[[('GBIF:4667268', 'Agustina', 0.7207658290863...","Counter({('GBIF:8121923', 'Taraxacum officinal...","[Counter({('GBIF:7424736', 'Lupinus nootkatens...","[(('GBIF:6', 'Plantae'), 52), (('GBIF:1', 'Ani...","[(('GBIF:7707728', 'Tracheophyta'), 52), (('GB...","[(('GBIF:220', 'Magnoliopsida'), 37), (('GBIF:...","[(('GBIF:414', 'Asterales'), 20), (('GBIF:408'...","[(('GBIF:3065', 'Asteraceae'), 20), (('GBIF:23...","[(('GBIF:7787708', 'Taraxacum'), 20), (('GBIF:...","[(('GBIF:5394163', 'Taraxacum officinale'), 15..."
1153,https://hal.inrae.fr/hal-02636091/document,"[[('GBIF:2438660', 'European Pine Vole', 0.730...","Counter({('GBIF:7812064', 'Pinus halepensis'):...","[Counter({('GBIF:2438660', 'European Pine Vole...","[(('GBIF:6', 'Plantae'), 216), (('GBIF:1', 'An...","[(('GBIF:770772

In [7]:
papers_df = pd.read_csv('C:/Users/maria/OneDrive/Documentos/Thesis/dataset_3/papers.csv', encoding='utf-8')
papers_df

,title_s,journalTitle_s,authLastName_s,authFullName_s,abstract_s,fileMain_s,ePublicationDateY_i,Complete,Genders,unknownGenders,content_text,cleaned_content_text,contentLen
0,"[""Exploring farmers' agrobiodiversity manageme...",People and Nature,"['Mariel', 'Carrière', 'Penot', 'Danthu', 'Raf...","['Juliette Mariel', 'Stéphanie M. Carrière', '...","[""1. Interactions between farmers and agrobiod...",https://hal.inrae.fr/hal-03461601/document,NaN,True,"['female', 'female', 'male', 'male', 'unknown'...",33.333333,HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,httpshalinraefrhal dec multidisciplinary open ...,48604
1,['Interpreting the mammal deposits of Cloggs C...,People and Nature,"['Mcdowell', 'David', 'Mullett', 'Fresløv', 'D...","['Matthew Mcdowell', 'Bruno David', 'Russell M...","[""Palaeontological animal bone deposits are ra...",https://hal.science/hal-03829927/document,NaN,True,"['male', 'male', 'male', 'female', 'unknown', ...",8.333333,HAL Id: hal-03829927 https://hal.science/hal-0...,httpshalsciencehal mar multidisciplinary open ...,47810
2,['An ecological theory of changing human popul...,People and Nature,"['Henderson', 'Loreau']","['Kirsten Henderson', 'Michel Loreau']",['The dependence of humans on nature has come ...,https://hal.science/hal-02350603/document,NaN,True,"['female', 'male']",0.000000,People and Nature. 2019;1:31–43. | 31 wileyonl...,people nature wileyonlinelibrarycomjournalpan ...,47819
3,"['Local knowledge, know‐how and knowledge mobi...",People and Nature,"['Congretel', 'Pinton']","['Mélanie Congretel', 'Florence F. Pinton']","[""More than 30 years after the 1988 Declaratio...",https://shs.hal.science/halshs-02937670/document,NaN,True,"['female', 'female']",0.000000,HAL Id: halshs-02937670 https://shs.hal.scienc...,halshs httpsshshalsciencehalshs multidisciplin...,68504
4,['From biodiversity to health: Quantifying the...,People and Nature,"['Ulrich', 'Batáry', 'Baudry', 'Beaumelle', 'B...","['Werner Ulrich', 'Péter Batáry', 'Julia Baudr...",['Ample evidence suggests positive effects of ...,https://cnam.hal.science/hal-04080723/document,NaN,True,"['male', 'male', 'female', 'female', 'male', '...",0.000000,People and Nature. 2023;5:69–83. | 69 wileyonl...,people nature wileyonlinelibrarycomjournalpan ...,58880
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,['Unidirectional response to bidirectional sel...,Ecology and Evolution,"['Renneville', 'Millot', 'Agostini', 'Carmigna...","['Clémentine Renneville', 'Alexis Millot', 'Si...",['Anthropogenic perturbations such as harvesti...,https://hal.sorbonne-universite.fr/hal-0295053...,NaN,True,"['female', 'male', 'male', 'male', 'female', '...",0.000000,HAL Id: hal-02950535 https://hal.sorbonne-univ...,httpshalsorbonneuniversitefrhal sep multidisci...,55877
1152,"['Think globally, measure locally: The MIREN s...",Ecology and Evolution,"['Haider', 'Lembrechts', 'Mcdougall', 'Pauchar...","['Sylvia Haider', 'Jonas J. Lembrechts', 'Keit...",['Climate change and other global change drive...,https://u-picardie.hal.science/hal-03619105/do...,NaN,True,"['female', 'male', 'male', 'male', 'male', 'fe...",13.793103,HAL Id: hal-03619105 https://u-picardie.hal.sc...,httpsupicardiehalsciencehal oct multidisciplin...,77580
1153,['Adaptation and plasticity in aboveground all...,Ecology and Evolution,"['Vizcaíno-Palomar', 'Ibáñez', 'González-Martí...","['Natalia Vizcaíno-Palomar', 'Inés Ibáñez', 'S...","[""Plant species aboveground allometry can be v...",https://hal.inrae.fr/hal-02636091/document,NaN,True,"['female', 'female', 'male', 'male', 'male']",0.000000,HAL Id: hal-02636091 https://hal.inrae.fr/hal-...,httpshalinraefrhal multidisciplinary open acce...,36497
1154,['Foraging and mating behaviors of Hypsignathu...,Ecology and Evolution,"['Schloesing', 'Caron', 'Chambon', 'Courbin', ...","['Elodie Schloesing', 'Alexandre Caron', 'Rémi...",['Studying wildlife space use in human-modifie...,https://hal.science/hal-04167917/document,2023.0,True,"['female', 